<B>This assumes it's being run from an active jupyter environment on the base image (docker run jupyter/base-notebook:latest")</B>
<p>Basic outline of differences: 
<p>1 - we move install to #1, ipywidgets won't come wiht the base image
<p>2 - imports
<p>3 - set your file path. environment comes with work/ folder by default
<p>4 - Define your converter functions (identical to main)
<p>5 - Do the thing (identical to main)

In [ ]:
#default image doesn't have ipywidgets
!pip install ipywidgets
!pip install torch
!pip install safetensors
!pip install pytorch-lightning

In [ ]:
#imports
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets
import os

In [ ]:
#upload PT files to your jupyter notebook instance, by default comes with "work" folder
#todo: add creation script, which is basically just "docker run jupyter/base-notebook:latest" and then grab the login token and URL
file_path = "work/" #@param {type:"string"}
verbose=True

In [ ]:
#Define Converter Functions
import os
from typing import Any, Dict

import torch
from safetensors.torch import save_file

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def process_pt_files(path: str, model_type: str, verbose=True) -> None:
    if os.path.isdir(path):
        # Path is a directory, process all .pt files in the directory
        for file_name in os.listdir(path):
            if file_name.endswith('.pt'):
                process_file(os.path.join(path, file_name), model_type, verbose)
    elif os.path.isfile(path) and path.endswith('.pt'):
        # Path is a .pt file, process this file
        process_file(path, model_type, verbose)
    else:
        print(f"{path} is not a valid directory or .pt file.")

def process_file(file_path: str, model_type: str, verbose: bool) -> None:
    # Load the PyTorch model
    model = torch.load(file_path, map_location=device)

    if verbose:
        print(file_path)

    if model_type == 'embedding':
        s_model = process_embedding_file(model, verbose)
    elif model_type == 'vae':
        s_model = process_vae_file(model, verbose)
    else:
        raise Exception(f"model_type `{model_type}` is not supported!")

    # Save the model with the new extension
    if file_path.endswith('.pt'):
        new_file_path = file_path[:-3] + '.safetensors'
    else:
        new_file_path = file_path + '.safetensors'
    save_file(s_model, new_file_path)

def process_embedding_file(model: Dict[str, Any], verbose: bool) -> Dict[str, torch.Tensor]:
    # Extract the embedding tensors
    model_tensors = model.get('string_to_param').get('*')
    s_model = {
          'emb_params': model_tensors
            }

    if verbose:
        # Print the requested training information, if it exists
        if ('sd_checkpoint_name' in model) and (model['sd_checkpoint_name'] is not None):
            print(f"Trained on {model['sd_checkpoint_name']}.")
        else:
            print("Checkpoint name not found in the model.")

        if ('step' in model) and (model['step'] is not None):
            print(f"Trained for {model['step']} steps.")
        else:
            print("Step not found in the model.")
        # Display the tensor's shape
        print(f"Dimensions of embedding tensor: {model_tensors.shape}")
        print()

    return s_model

def process_vae_file(model: Dict[str, Any], verbose: bool) -> Dict[str, torch.Tensor]:
    # Extract the state dictionary
    s_model = model["state_dict"]
    if verbose:
        # Print the requested training information, if it exists
        step = model.get('step', model.get('global_step'))
        if step is not None:
            print(f"Trained for {step} steps.")
        else:
            print("Step not found in the model.")
        print()
    return s_model

In [ ]:
#@title Convert the Embedding(s)
process_pt_files(file_path, 'embedding', verbose=verbose)

In [ ]:
#@title Convert the VAE(s)
process_pt_files(file_path, 'vae', verbose=verbose)